In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from hloc import (
    extract_features,
    match_features,
    reconstruction,
    visualization,
    pairs_from_retrieval,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup
In this notebook, we will run SfM reconstruction from scratch on a set of images. We choose the [South-Building dataset](https://openaccess.thecvf.com/content_cvpr_2013/html/Hane_Joint_3D_Scene_2013_CVPR_paper.html) - we will download it later. First, we define some paths.

In [3]:
dataset = Path("../data/train/dioscuri/")  # change this if your dataset is somewhere else
images = dataset / "images/"

outputs = Path("outputs/sfm/")
sfm_pairs = outputs / "pairs-dinov2.txt"
sfm_dir = outputs / "sfm_superpoint+superglue"

retrieval_conf = extract_features.confs["dinov2_salad"]
feature_conf = extract_features.confs["xfeat"]
matcher_conf = match_features.confs["NN-xfeat"]

## Find image pairs via image retrieval
We extract global descriptors with NetVLAD and find for each image the most similar ones. For smaller dataset we can instead use exhaustive matching via `hloc/pairs_from_exhaustive.py`, which would find $\frac{n(n-1)}{2}$ images pairs.

In [4]:
retrieval_path = extract_features.main(retrieval_conf, images, outputs)
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=5)

[2024/05/19 19:21:18 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'dinov2_salad'},
 'output': 'global-feats-dinov2salad',
 'preprocessing': {'resize_max': 1024}}
[2024/05/19 19:21:18 hloc INFO] Found 70 images in root ../data/train/dioscuri/images.
Using cache found in /home/salva/.cache/torch/hub/serizba_salad_main
Using cache found in /home/salva/.cache/torch/hub/facebookresearch_dinov2_main
/home/salva/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/salva/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/salva/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
  0%|          | 0/70 

## Extract and match local features

In [5]:
feature_path = extract_features.main(feature_conf, images, outputs)
match_path = match_features.main(
    matcher_conf, sfm_pairs, feature_conf["output"], outputs
)

[2024/05/19 19:21:58 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'xfeat', 'top_k': 4096},
 'output': 'feats-xfeat',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}
[2024/05/19 19:21:58 hloc INFO] Found 70 images in root ../data/train/dioscuri/images.
Using cache found in /home/salva/.cache/torch/hub/verlab_accelerated_features_main
100%|██████████| 70/70 [00:03<00:00, 23.09it/s]
[2024/05/19 19:22:02 hloc INFO] Finished exporting features.
[2024/05/19 19:22:02 hloc INFO] Matching local features with configuration:
{'model': {'name': 'cosine_mlp', 'top_k': 4096},
 'output': 'matches-xfeat',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}
Using cache found in /home/salva/.cache/torch/hub/verlab_accelerated_features_main
100%|██████████| 217/217 [00:02<00:00, 74.80it/s]
[2024/05/19 19:22:05 hloc INFO] Finished exporting matches.


## 3D reconstruction
Run COLMAP on the features and matches.

In [6]:
model = reconstruction.main(sfm_dir, images, sfm_pairs, feature_path, match_path)

[2024/05/19 19:22:14 hloc INFO] Creating an empty database...
[2024/05/19 19:22:14 hloc INFO] Importing images into the database...
[2024/05/19 19:22:15 hloc INFO] Importing features into the database...
100%|█████████████████████████████████████████| 70/70 [00:00<00:00, 1523.75it/s]
[2024/05/19 19:22:15 hloc INFO] Importing matches into the database...
  0%|                                                   | 0/350 [00:00<?, ?it/s]


IndexError: index 386 is out of bounds for axis 0 with size 386

## Visualization
We visualize some of the registered images, and color their keypoint by visibility, track length, or triangulated depth.

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by="visibility", n=5)

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by="track_length", n=5)

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by="depth", n=5)